# API Status Shortcuts

This notebook provides a few quick ways to visualize the current outputs of the Fire Events Data Suite Near-Real Time API (FEDS NRT API). Information about the FEDS data and project can be found on the team's website [here.](https://earth-information-system.github.io/fireatlas/docs/)


Links: 

[Fire Event Explorer](https://earthdata.nasa.gov/dashboard/tools/fire-event-explorer) for North America visualization. 

[FIRMS US/Canada](https://firms.modaps.eosdis.nasa.gov/usfs/map/#m:experimental;d:24hrs,24hrs;l:fires_viirs_noaa20,fires_viirs_noaa21,fires_viirs_snpp,eis_fire_lf_perimeter_nrt,countries,earth;@-102.8,37.6,5.1z) for alternate North America visualization and to compare with raw satellite input and NIFC reported fires. 


See below for a few quick visual checks to see what the API is currently serving in terms of NRT data. 

In [ ]:
!pip install owslib==0.28.1 -q

In [ ]:
import owslib
owslib.__version__

In [ ]:
import lonboard 
import geopandas as gpd 
import datetime as dt
from owslib.ogcapi.features import Features 

OGC_URL = "https://openveda.cloud/api/features"
api = Features(url=OGC_URL)

print("Available collections:\n")
for c in api.feature_collections():
    try:
        # New API: infer temporal extent by sorting on property 't'
        newest = api.collection_items(c, **{"sortby": "-t"}, limit=1)
        oldest = api.collection_items(c, **{"sortby": "t"}, limit=1)
        t_min = oldest["features"][0]["properties"].get("t") if oldest.get("features") else None
        t_max = newest["features"][0]["properties"].get("t") if newest.get("features") else None
        if t_min and t_max:
            print(f"{c}: {t_min.split('T')[0]} to {t_max.split('T')[0]}")
        else:
            print(c)
    except Exception as e:
        print(c)


## Largefire collections for last N days 

Different from the snapshot filtering above: lf_nrt collections include all timesteps for that fire. 

Pulls from the following collections: 

```
eis_fire_lf_perimeter_nrt
eis_fire_lf_fireline_nrt
eis_fire_lf_newfirepix_nrt
```

In [ ]:
# Setup dates 
N = 3 # get data from last N days 

now = dt.datetime.now(dt.timezone.utc)
start = now - dt.timedelta(days=N)

start = dt.datetime.strftime(start, "%Y-%m-%dT%H:%M:%S+00:00")
stop = dt.datetime.strftime(now, "%Y-%m-%dT%H:%M:%S+00:00")

print(start, stop)

# query API 

perims = api.collection_items(
    "public.eis_fire_lf_perimeter_nrt",
    bbox=[],
    datetime=[start + "/" + stop],
    limit=8000
)

assert(perims["numberMatched"] == perims["numberReturned"])
print(f"{perims['numberReturned']} perimeters returned")

flines = api.collection_items(
    "public.eis_fire_lf_fireline_nrt",
    bbox=[],
    datetime=[start + "/" + stop],
    limit=8000
)

assert(flines["numberMatched"] == flines["numberReturned"])
print(f"{flines['numberReturned']} firelines returned")
fpix = api.collection_items(
    "public.eis_fire_lf_newfirepix_nrt",
    bbox=[],
    datetime=[start + "/" + stop],
    limit=8000
)

assert(fpix["numberMatched"] == fpix["numberReturned"])
print(f"{fpix['numberReturned']} pixel detections returned.")

# Check if we got any data back
got_perims = perims.get('numberReturned', 0) > 0
got_flines = flines.get('numberReturned', 0) > 0
got_fpix = fpix.get('numberReturned', 0) > 0
if not (got_perims or got_flines or got_fpix):
    raise RuntimeError('No perimeter, fireline, or fire pixel features were returned.')


perims_gdf = gpd.GeoDataFrame.from_features(perims["features"]).set_crs("epsg:4326")
flines_gdf = None
if flines.get('features'):
    flines_gdf = gpd.GeoDataFrame.from_features(flines['features']).set_crs('epsg:4326')
fpix_gdf = None
if fpix.get('features'):
    fpix_gdf = gpd.GeoDataFrame.from_features(fpix['features']).set_crs('epsg:4326')

In [ ]:
# Map with lonboard 

# perimeter layer 
perims_layer = lonboard.PolygonLayer.from_geopandas(
    perims_gdf, 
    filled=False, 
    stroked=True, 
    get_fill_color='aqua',
    get_line_color='aqua',
    opacity=.05, 
    line_width_min_pixels=1
)

# fireline layer 
flines_layer = lonboard.PathLayer.from_geopandas(
    flines_gdf, 
    get_color='red', 
    width_min_pixels=1, 
    width_scale=50
)

# fire pixel layer 
fpix_layer = lonboard.ScatterplotLayer.from_geopandas(
    fpix_gdf, 
    get_fill_color='red', 
    filled=True, 
    radius_scale=50, 
    radius_min_pixels=1
)

m = lonboard.Map([perims_layer, fpix_layer, flines_layer], basemap_style=lonboard.basemap.CartoBasemap.DarkMatterNoLabels)
m

## Last N days of NRT perimeters, firelines, and newfirepix

Pulls from the following collections: 

```
eis_fire_snapshot_perimeter_nrt
eis_fire_snapshot_fireline_nrt
eis_fire_snapshot_newfirepix_nrt
```

Note that these are NOT the large fire collections, but the snapshot collections, and are not guaranteed to go back more than 20 days. 

In [ ]:
N = 7 # get data from last N days 

now = dt.datetime.now(dt.timezone.utc)
start = now - dt.timedelta(days=N)

start = dt.datetime.strftime(start, "%Y-%m-%dT%H:%M:%S+00:00")
stop = dt.datetime.strftime(now, "%Y-%m-%dT%H:%M:%S+00:00")

print(start, stop)

# query API 

perims = api.collection_items(
    "public.eis_fire_snapshot_perimeter_nrt",
    bbox=[],
    datetime=[start + "/" + stop],
    limit=8000
)

assert(perims["numberMatched"] == perims["numberReturned"])
print(f"{perims['numberReturned']} perimeters returned")

flines = api.collection_items(
    "public.eis_fire_snapshot_fireline_nrt",
    bbox=[],
    datetime=[start + "/" + stop],
    limit=8000
)

assert(flines["numberMatched"] == flines["numberReturned"])
print(f"{flines['numberReturned']} firelines returned")
fpix = api.collection_items(
    "public.eis_fire_snapshot_newfirepix_nrt",
    bbox=[],
    datetime=[start + "/" + stop],
    limit=8000
)

assert(fpix["numberMatched"] == fpix["numberReturned"])
print(f"{fpix['numberReturned']} pixel detections returned.")

# Check if we got any data back
got_perims = perims.get('numberReturned', 0) > 0
got_flines = flines.get('numberReturned', 0) > 0
got_fpix = fpix.get('numberReturned', 0) > 0
if not (got_perims or got_flines or got_fpix):
    raise RuntimeError('No perimeter, fireline, or fire pixel features were returned.')

perims_gdf = gpd.GeoDataFrame.from_features(perims["features"]).set_crs("epsg:4326")
flines_gdf = None
if flines.get('features'):
    flines_gdf = gpd.GeoDataFrame.from_features(flines['features']).set_crs('epsg:4326')
fpix_gdf = None
if fpix.get('features'):
    fpix_gdf = gpd.GeoDataFrame.from_features(fpix['features']).set_crs('epsg:4326')

In [ ]:
# Map with lonboard 

# perimeter layer 
perims_layer = lonboard.PolygonLayer.from_geopandas(
    perims_gdf, 
    filled=True, 
    stroked=False, 
    get_fill_color='aqua',
    opacity=.05
)

if flines_gdf is not None and fpix_gdf is not None: 
    flines_layer = lonboard.PathLayer.from_geopandas(
        flines_gdf, 
        get_color='red', 
        width_min_pixels=1, 
        width_scale=50
    )

    # fire pixel layer 
    fpix_layer = lonboard.ScatterplotLayer.from_geopandas(
        fpix_gdf, 
        get_fill_color='red', 
        filled=True, 
        radius_scale=50, 
        radius_min_pixels=1
    )
    
    m = lonboard.Map([perims_layer, fpix_layer, flines_layer], basemap_style=lonboard.basemap.CartoBasemap.DarkMatterNoLabels)
else: 
    m = lonboard.Map(perims_layer, basemap_style=lonboard.basemap.CartoBasemap.DarkMatterNoLabels)
m

## Large fires (>5km^2) in last N days, filtered from Snapshot collections

Pulls from the following collections: 

```
eis_fire_snapshot_perimeter_nrt
eis_fire_snapshot_fireline_nrt
eis_fire_snapshot_newfirepix_nrt
```

Note that these only show the most recent perimeter etc., whereas the `lf_nrt` collections show the history of each large fire across multiple timesteps. 

This view is much snappier for viewing the latest products, but does not show evolution at a glance. 

In [ ]:
# Setup dates 
N = 3 # get data from last N days 

now = dt.datetime.now(dt.timezone.utc)
start = now - dt.timedelta(days=N)

start = dt.datetime.strftime(start, "%Y-%m-%dT%H:%M:%S+00:00")
stop = dt.datetime.strftime(now, "%Y-%m-%dT%H:%M:%S+00:00")

print(start, stop)

# query API 

perims = api.collection_items(
    "public.eis_fire_snapshot_perimeter_nrt",
    bbox=[],
    datetime=[start + "/" + stop],
    limit=8000, 
    filter="farea>5"
)

assert(perims["numberMatched"] == perims["numberReturned"])
print(f"{perims['numberReturned']} perimeters returned")

perims_gdf = gpd.GeoDataFrame.from_features(perims["features"]).set_crs("epsg:4326")
largefire_ids = perims_gdf.fireid.unique()
largefire_ids = ",".join(map(str, largefire_ids)) # make string for query

flines = api.collection_items(
    "public.eis_fire_snapshot_fireline_nrt",
    bbox=[],
    datetime=[start + "/" + stop],
    limit=8000, 
    filter="fireid IN (" + largefire_ids + ")"
)

assert(flines["numberMatched"] == flines["numberReturned"])
print(f"{flines['numberReturned']} firelines returned")
fpix = api.collection_items(
    "public.eis_fire_snapshot_newfirepix_nrt",
    bbox=[],
    datetime=[start + "/" + stop],
    limit=8000,
    filter="fireid IN (" + largefire_ids + ")"
)

assert(fpix["numberMatched"] == fpix["numberReturned"])
print(f"{fpix['numberReturned']} pixel detections returned.")

# Check if we got any data back
got_perims = perims.get('numberReturned', 0) > 0
got_flines = flines.get('numberReturned', 0) > 0
got_fpix = fpix.get('numberReturned', 0) > 0
if not (got_perims or got_flines or got_fpix):
    raise RuntimeError('No perimeter, fireline, or fire pixel features were returned.')



flines_gdf = None
if flines.get('features'):
    flines_gdf = gpd.GeoDataFrame.from_features(flines['features']).set_crs('epsg:4326')
fpix_gdf = None
if fpix.get('features'):
    fpix_gdf = gpd.GeoDataFrame.from_features(fpix['features']).set_crs('epsg:4326')

In [ ]:
# Map with lonboard 

# perimeter layer 
perims_layer = lonboard.PolygonLayer.from_geopandas(
    perims_gdf, 
    filled=True, 
    stroked=False, 
    get_fill_color='aqua',
    opacity=.05
)

if flines_gdf is not None and fpix_gdf is not None: 
    # fireline layer 
    flines_layer = lonboard.PathLayer.from_geopandas(
        flines_gdf, 
        get_color='red', 
        width_min_pixels=1, 
        width_scale=50
    )
    
    # fire pixel layer 
    fpix_layer = lonboard.ScatterplotLayer.from_geopandas(
        fpix_gdf, 
        get_fill_color='red', 
        filled=True, 
        radius_scale=50, 
        radius_min_pixels=1
    )
    
    m = lonboard.Map([perims_layer, fpix_layer, flines_layer], basemap_style=lonboard.basemap.CartoBasemap.DarkMatterNoLabels)
else: 
    m = lonboard.Map(perims_layer, basemap_style=lonboard.basemap.CartoBasemap.DarkMatterNoLabels)

m

## Viz history of largest fires (>100km^2) for past N days 

Pulls from the following collections: 

```
eis_fire_lf_perimeter_nrt
eis_fire_lf_fireline_nrt
eis_fire_lf_newfirepix_nrt
```

In [ ]:
# Setup dates 
N = 30 # get data from last N days 

now = dt.datetime.now(dt.timezone.utc)
start = now - dt.timedelta(days=N)

start = dt.datetime.strftime(start, "%Y-%m-%dT%H:%M:%S+00:00")
stop = dt.datetime.strftime(now, "%Y-%m-%dT%H:%M:%S+00:00")

print(start, stop)

# query API 

perims = api.collection_items(
    "public.eis_fire_lf_perimeter_nrt",
    bbox=[],
    datetime=[start + "/" + stop],
    limit=8000, 
    filter="farea>100" # AREA THRESHOLD
)

assert(perims["numberMatched"] == perims["numberReturned"])
print(f"{perims['numberReturned']} perimeters returned")

if perims['numberReturned'] > 1: 
    perims_gdf = gpd.GeoDataFrame.from_features(perims["features"]).set_crs("epsg:4326")
    largefire_ids = perims_gdf.fireid.unique()
    largefire_ids = ",".join(map(str, largefire_ids)) # make string for query
    
    flines = api.collection_items(
        "public.eis_fire_lf_fireline_nrt",
        bbox=[],
        datetime=[start + "/" + stop],
        limit=8000, 
        filter="fireid IN (" + largefire_ids + ")"
    )
    
    assert(flines["numberMatched"] == flines["numberReturned"])
    print(f"{flines['numberReturned']} firelines returned")
    fpix = api.collection_items(
        "public.eis_fire_lf_newfirepix_nrt",
        bbox=[],
        datetime=[start + "/" + stop],
        limit=8000,
        filter="fireid IN (" + largefire_ids + ")"
    )
    
    assert(fpix["numberMatched"] == fpix["numberReturned"])
    print(f"{fpix['numberReturned']} pixel detections returned.")

# Check if we got any data back
got_perims = perims.get('numberReturned', 0) > 0
got_flines = flines.get('numberReturned', 0) > 0
got_fpix = fpix.get('numberReturned', 0) > 0
if not (got_perims or got_flines or got_fpix):
    raise RuntimeError('No perimeter, fireline, or fire pixel features were returned.')

    
    flines_gdf = None
if flines.get('features'):
    flines_gdf = gpd.GeoDataFrame.from_features(flines['features']).set_crs('epsg:4326')
    fpix_gdf = None
if fpix.get('features'):
    fpix_gdf = gpd.GeoDataFrame.from_features(fpix['features']).set_crs('epsg:4326')
else: 
    print(f"Warning: 0 very large fires (>100km2) found for {start} to {stop}")
    perims_gdf = None

In [ ]:
# Map with lonboard 

if perims_gdf: 
    # perimeter layer 
    perims_layer = lonboard.PolygonLayer.from_geopandas(
        perims_gdf, 
        filled=False, 
        stroked=True, 
        get_fill_color='aqua',
        get_line_color='aqua',
        opacity=.05, 
        line_width_min_pixels=1
    )
    
    # fireline layer 
    flines_layer = lonboard.PathLayer.from_geopandas(
        flines_gdf, 
        get_color='red', 
        width_min_pixels=1, 
        width_scale=50
    )
    
    # fire pixel layer 
    fpix_layer = lonboard.ScatterplotLayer.from_geopandas(
        fpix_gdf, 
        get_fill_color='red', 
        filled=True, 
        radius_scale=50, 
        radius_min_pixels=1
    )
    
    m = lonboard.Map([perims_layer, fpix_layer, flines_layer], basemap_style=lonboard.basemap.CartoBasemap.DarkMatterNoLabels)
    m